<a href="https://colab.research.google.com/github/Sanchitakr/Bert_qna_scoring/blob/main/Bert_bertscore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [18]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [19]:
!pip install transformers


In [20]:
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer,BertForMaskedLM

#Model
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

#masks
bertMaskedLM = BertForMaskedLM.from_pretrained('bert-base-uncased')

#Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [21]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

tokenizer = AutoTokenizer.from_pretrained("deepset/bert-large-uncased-whole-word-masking-squad2")

model = AutoModelForQuestionAnswering.from_pretrained("deepset/bert-large-uncased-whole-word-masking-squad2")

In [ ]:
questions='''What is your name and DOB?',

'What bring you here today?',

'How did it start?',

'Are you taking any pain killers?',

'How often do you take it?',

'Are you on any other medications?',

'Do you think bright light affect it?',

'How do you rate it?',

'Do you sleep well?',

'Do you feel like vomitting?',

'Does the headache make you restless?',

'Are there any ear problems?',

'Do you feel sweat?',

'What do you think has caused it?',

'Do you feel any memory issues?',

'How is your diet?',

'How is your appetite?"'''

for ques in range(len(questions)):
  question = (questions[ques])

In [ ]:
qbfile = open("q.txt", "r")

for aline in qbfile:
    values=aline.rstrip('\n')
    values = aline.split()
    qt=(' '.join(values))
qbfile.close()


In [ ]:
def get_score(sentence):
    tokenize_input = tokenizer.tokenize(sentence)
    tensor_input = torch.tensor([tokenizer.convert_tokens_to_ids(tokenize_input)])
    predictions=bertMaskedLM(tensor_input)
    loss_fct = torch.nn.CrossEntropyLoss()
    loss = loss_fct(predictions.squeeze(),tensor_input.squeeze()).data 
    return math.exp(loss)

['How', 'is', 'your', 'appetite?']


In [ ]:
question = '''does the pain travel anywhere else?
"'''

paragraph = ''' My name is Jenny Taylor and my date of birth is 22nd March.I've been experiencing really bad headaches recently. These headaches usually starts in evening and last for 4-6 hours.I had this 2 times within last 6 months but I have never seen a doctor. I get these headaches only on the left side of the head. 
I've already taken Panadol but it's not doing much for my pain. It'd be good to have more pain relief if that's possible please.I have been taking 2 tablets of Panadol 4 times a day for 2 days for this particular headache.I am not on any medication aside from the panadol for the headaches.The pain started while I was in middle of doing some gardening in my backyard.
My headache is pulsating. The pain seems to travel or move to the back of my head.I do not feel tired when the pain starts however the headache affects my daily routine like gardening. I feel like vomitting but I actually don't when I get the headache or pain.
The bright lights affect my pain a lot, and I feel if I avoid loud sounds, I feel better.I do not have any issues with my sleep, my memory and never had any fits.No one has noticed any personality changes. The headache does not make me feel agitated or restless. 
I don't have runny nose, no breathing issues, no sneezing, no cough, no blocked nose and does not feel any pain in and around chest.I don't have any palpitations or numbness and don't have any ear problem like bleeding or discharge from ear, ear pain etc.I have not had any dizziness, drowsiness and neither have I experienced any fainting spells. I do not have any rashes.
There is no itching. However, my vision become blurry just before the headache starts but my eye sight is normal otherwise.I don't have any neck, jaw or pain anywhere else in my body. I never had any abortion or miscarriages.I have never been diagnosed with TB, Thyroid, Epilepsy, High BP, Diabetes and I have never got done any surgery before.My apetite has been fine and there are no issues with my diet or weight. I feel normal and does not sweat or shiver much.
The pain severity is 7 out of 10.'''
            
encoding = tokenizer.encode_plus(text=question,text_pair=paragraph, special_tokens=["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"],max_len=543,padding='max_length',wordpieces_prefix="##")

inputs = encoding['input_ids']  #Token embeddings
sentence_embedding = encoding['token_type_ids']  #Segment embeddings
tokens = tokenizer.convert_ids_to_tokens(inputs) #input tokens

Keyword arguments {'special_tokens': ['[PAD]', '[UNK]', '[CLS]', '[SEP]', '[MASK]'], 'max_len': 543, 'wordpieces_prefix': '##'} not recognized.
Keyword arguments {'special_tokens': ['[PAD]', '[UNK]', '[CLS]', '[SEP]', '[MASK]'], 'max_len': 543, 'wordpieces_prefix': '##'} not recognized.


In [ ]:
encoding['token_type_ids']

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [ ]:
questions=['What is your name and DOB?',

'What bring you here today?',

'How did it start?',

'Are you taking any pain killers?',

'How often do you take it?',

'Are you on any other medications?',

'Do you think bright light affect it?',

'How do you rate it?',

'Do you sleep well?',

'Do you feel like vomitting?',

'Does the headache make you restless?',

'Are there any ear problems?',

'Do you feel sweat?',

'What do you think has caused it?',

'Do you feel any memory issues?',

'How is your diet?',

'How is your appetite?']
question=[]
for i,ques in enumerate(questions):
  question=ques

print("DONE!")
  


DONE!


In [ ]:
import csv
input=['What is your name and DOB?',

'What bring you here today?',

'How did it start?',

'Are you taking any pain killers?',

'How often do you take it?',

'Are you on any other medications?',

'Do you think bright light affect it?',

'How do you rate it?',

'Do you sleep well?',

'Do you feel like vomitting?',

'Does the headache make you restless?',

'Are there any ear problems?',

'Do you feel sweat?',

'What do you think has caused it?',

'Do you feel any memory issues?',

'How is your diet?',

'How is your appetite?"']
#input = ['abc,"a string, with a comma","another, one"']
parser = csv.reader(input)

for fields in parser:
  for i,f in enumerate(fields):
    print (f)    # in Python 3 and up, print is a function; use: print(i,f)

What is your name and DOB?
What bring you here today?
How did it start?
Are you taking any pain killers?
How often do you take it?
Are you on any other medications?
Do you think bright light affect it?
How do you rate it?
Do you sleep well?
Do you feel like vomitting?
Does the headache make you restless?
Are there any ear problems?
Do you feel sweat?
What do you think has caused it?
Do you feel any memory issues?
How is your diet?
How is your appetite?"


In [ ]:
quest

['What is your name and DOB?',
 'What bring you here today?',
 'How did it start?',
 'Are you taking any pain killers?',
 'How often do you take it?',
 'Are you on any other medications?',
 'Do you think bright light affect it?',
 'How do you rate it?',
 'Do you sleep well?',
 'Do you feel like vomitting?',
 'Does the headache make you restless?',
 'Are there any ear problems?',
 'Do you feel sweat?',
 'What do you think has caused it?',
 'Do you feel any memory issues?',
 'How is your diet?']

In [ ]:
#exceeds the token limit after adding these 2 lines [WHY?] 
I am not sure what has caused or triggered the headache but the pain is slowly getting worse. The pain severity is 7 out of 10. 
My periods are normal and generally lasts 4-5 days. My blood group is O-positive.

In [ ]:
print(len(encoding['input_ids']))

#print(tokens)
print(len(encoding['token_type_ids']))
decoded_sequence = tokenizer.decode(inputs)
print(decoded_sequence)

512
512
[CLS] does the pain travel anywhere else? " [SEP] my name is jenny taylor and my date of birth is 22nd march. i've been experiencing really bad headaches recently. these headaches usually starts in evening and last for 4 - 6 hours. i had this 2 times within last 6 months but i have never seen a doctor. i get these headaches only on the left side of the head. i've already taken panadol but it's not doing much for my pain. it'd be good to have more pain relief if that's possible please. i have been taking 2 tablets of panadol 4 times a day for 2 days for this particular headache. i am not on any medication aside from the panadol for the headaches. the pain started while i was in middle of doing some gardening in my backyard. my headache is pulsating. the pain seems to travel or move to the back of my head. i do not feel tired when the pain starts however the headache affects my daily routine like gardening. i feel like vomitting but i actually don't when i get the headache or pai

In [ ]:
start_scores, end_scores = model(input_ids=torch.tensor([inputs]), token_type_ids=torch.tensor([sentence_embedding]))


In [ ]:
# get the position of the masked token
masked_position = tokenizer.mask_token_id
print(masked_position)

103


In [ ]:
start_index = torch.argmax(start_scores)

end_index = torch.argmax(end_scores)

answer = ' '.join(tokens[start_index:end_index+1])

In [ ]:
corrected_answer = ''

for word in answer.split():
    
    #If it's a subword token
    if word[0:2] == '##':
        corrected_answer += word[2:]
    else:
        corrected_answer += ' ' + word

In [ ]:
corrected_answer

' the pain seems to travel or move to the back of my head'

In [22]:
!pip install bert_score==0.3.4

In [23]:
from bert_score import score

In [24]:


with open("/content/q.txt") as f:
    cands = [line.strip() for line in f]

with open("/content/refs.txt") as f:
    refs = [line.strip() for line in f]

In [25]:
refs[0]


'If you have any sweating sessions?.'

In [26]:
# When you are running this cell for the first time, 
# it will download the BERT model which will take relatively longer. 
P, R, F1 = score(cands, refs, lang="en", verbose=True)

calculating scores...
computing bert embedding.


/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1324: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was n


computing greedy matching.



done in 0.18 seconds, 51.15 sentences/sec


In [30]:
print("Sentence:",refs[0],"score:",F1[0])
print("Sentence:",refs[1],"score:",F1[1])
print("Sentence:",refs[2],"score:",F1[2])
print("Sentence:",refs[3],"score:",F1[3])
print("Sentence:",refs[4],"score:",F1[4])
print("Sentence:",refs[5],"score:",F1[5])

Sentence: If you have any sweating sessions?. score: tensor(0.9721)
Sentence: Were you sweating?. score: tensor(0.9828)
Sentence: Are you sweating? score: tensor(0.9664)
Sentence: Do you sweat more than usual?. score: tensor(0.9824)
Sentence: Did you sweat?. score: tensor(0.9817)
Sentence: Are you going to sweat?. score: tensor(0.9570)
